In [47]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

gr.close_all()

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

MODEL = "gpt-4o-mini"
openai = OpenAI()

# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "athens": "€300"}
ticket_availability = {"london": "yes", "paris": "limited", "tokyo": "low", "berlin": "high", "athens": "No"}
ticket_weather = {"athens": "36", "paris": "26", "tokyo": "15", "berlin": "22"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def get_ticket_availability(destination_city):
    print(f"Tool get_ticket_availability called for {destination_city}")
    city = destination_city.lower()
    return ticket_availability.get(city, "Unknown")

def get_weather(destination_city):
    print(f"Tool get_weather called for {destination_city}")
    city = destination_city.lower()
    return ticket_weather.get(city, "Unknown")

# Define your tools (functions)
weather_function = {
    "name": "get_weather",
    "description": "Returns the current weather for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The name of the city to get the weather for"
            }
        },
        "required": ["destination_city"]
    }
}

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

availability_function = {
    "name": "get_ticket_availability",
    "description": "Get the ticket availabilty level to the destination city. Call this whenever you need to know the ticket availability, for example when a customer asks 'Are there any available tickets for this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# And this is included in a list of tools:
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": availability_function},
    {"type": "function", "function": weather_function}
]

# Dummy tool call handler
def handle_tool_call(tool_call):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    destination_city = args.get("destination_city", "unknown")

    if name == "get_ticket_price":
        response = get_ticket_price(destination_city)
    elif name == "get_ticket_availability":
        response = get_ticket_availability(destination_city)
    elif name == "get_weather":
        response = get_weather(destination_city)
    else:
        response = "Unknown tool."

    return response

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

# Chat function with multiple tool call support
def chat(message, history):
    smessage = {"role": "system", "content": system_message}
    messages = [smessage] + history + [{"role": "user", "content": message}]
    
    # First model call
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    reply = response.choices[0].message
    # If tool calls are requested
    if response.choices[0].finish_reason=="tool_calls":
        tool_calls = reply.tool_calls

        for tool_call in tool_calls:
            tool_response = handle_tool_call(tool_call)
            print(tool_response)
            # Append tool call
            messages.append({
                "role": "assistant",
                "tool_call_id": tool_call.id,
                "content": None,
                "tool_calls": [tool_call]
            })

            # Append tool response
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": tool_response
            })

        # Second model call with tool responses
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )

    return response.choices[0].message.content

In [48]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7896
* To create a public link, set `share=True` in `launch()`.


[ChatCompletionMessageToolCall(id='call_1MEVC5OzAGZQ3vwhBWphsSD1', function=Function(arguments='{"destination_city": "Tokyo"}', name='get_ticket_price'), type='function'), ChatCompletionMessageToolCall(id='call_VQPKIEGCZsp9zYQ6JigcSKAf', function=Function(arguments='{"destination_city": "Tokyo"}', name='get_ticket_availability'), type='function'), ChatCompletionMessageToolCall(id='call_sUGZSMmF5rJI3pHfTuHwoa3G', function=Function(arguments='{"destination_city": "Tokyo"}', name='get_weather'), type='function')]
Tool get_ticket_price called for Tokyo
$1400
Tool get_ticket_availability called for Tokyo
low
Tool get_weather called for Tokyo
15
[ChatCompletionMessageToolCall(id='call_EumVPj9PazrQzFUXUm7XAuSn', function=Function(arguments='{"destination_city": "Athens"}', name='get_weather'), type='function'), ChatCompletionMessageToolCall(id='call_8wWMxmVdmEpHHxbwkkGJmIMV', function=Function(arguments='{"destination_city": "Athens"}', name='get_ticket_availability'), type='function'), ChatC

In [33]:
get_ticket_availability("Paris")

Tool get_ticket_availability called for Paris


'limited'

In [24]:
get_ticket_availability("Athens")

Tool get_ticket_availability called for Athens


'No'

In [34]:
get_ticket_price("Athens")

Tool get_ticket_price called for Athens


'€300'

In [35]:
get_weather("Athens")

Tool get_weather called for Athens


'36'